In [93]:
cd /content/drive/MyDrive/EAM/Especializacion/Analitica predictiva/Curso EAM 4/Carpeta del Curso Analítica Predictiva/Para despliegue - Aprobacion curso

/content/drive/MyDrive/EAM/Especializacion/Analitica predictiva/Curso EAM 4/Carpeta del Curso Analítica Predictiva/Para despliegue - Aprobacion curso


In [94]:
ls

'Aprobacion curso 2019 fut.xlsx'   despliegue.ipynb
'Aprobacion curso 2019.xlsx'       requirements.txt
 best_stacking_model.joblib


In [95]:
import pandas as pd

df = pd.read_excel('Aprobacion curso 2019 fut.xlsx', sheet_name='Estudiantes 2019-01')
display(df.head())

,ID,Año - Semestre,Felder,Examen_admisión
0,161,2019-01,activo,4.00
1,162,2019-01,visual,3.67
2,163,2019-01,equilibrio,4.50
3,164,2019-01,visual,3.00
4,165,2019-01,equilibrio,4.67


In [96]:
df = df.drop(['ID', 'Año - Semestre'], axis=1)
display(df.head())

,Felder,Examen_admisión
0,activo,4.00
1,visual,3.67
2,equilibrio,4.50
3,visual,3.00
4,equilibrio,4.67


In [97]:
import joblib

onehot_encoder = joblib.load('onehot_encoder.joblib')

In [98]:
# Convert the one-hot encoded array to a DataFrame
# We need to get the feature names from the onehot_encoder
felder_feature_names = onehot_encoder.get_feature_names_out(['Felder'])
felder_encoded_df = pd.DataFrame(felder_encoded, columns=felder_feature_names)

# Concatenate the original DataFrame (without the original 'Felder' column) and the one-hot encoded DataFrame
df = pd.concat([df, felder_encoded_df], axis=1)

# Drop the original 'Felder' column from df
df = df.drop('Felder', axis=1)

display(df.head())

,Examen_admisión,Felder_activo,Felder_equilibrio,Felder_intuitivo,Felder_reflexivo,Felder_secuencial,Felder_sensorial,Felder_verbal,Felder_visual
0,4.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.67,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
import joblib

minmax_scaler = joblib.load('minmax_scaler.joblib')

In [100]:
# Apply the scaler to the 'Examen_admisión' column
df['Examen_admisión'] = minmax_scaler.transform(df[['Examen_admisión']])

display(df.head())

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


,Examen_admisión,Felder_activo,Felder_equilibrio,Felder_intuitivo,Felder_reflexivo,Felder_secuencial,Felder_sensorial,Felder_verbal,Felder_visual
0,4.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.67,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
import joblib

best_stacking_model = joblib.load('best_stacking_model.joblib')

In [102]:
import numpy as np

# Combine the scaled numerical feature and the one-hot encoded categorical features
X_future = np.hstack((df[['Examen_admisión']].values, felder_encoded))

# Make predictions using the loaded model
predictions = best_stacking_model.predict(X_future)

# Display the predictions
print("Predictions:")
display(predictions)

Predictions:


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array(['si', 'si', 'si', 'si', 'si', 'si', 'si', 'si'], dtype=object)

In [103]:
%pip install streamlit pandas openpyxl scikit-learn

In [104]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np

st.title('Predicción de Aprobación de Curso')

# Load the trained model and preprocessing objects
try:
    onehot_encoder = joblib.load('onehot_encoder.joblib')
    minmax_scaler = joblib.load('minmax_scaler.joblib')
    best_stacking_model = joblib.load('best_stacking_model.joblib')
except FileNotFoundError:
    st.error("Error: Make sure 'onehot_encoder.joblib', 'minmax_scaler.joblib', and 'best_stacking_model.joblib' are in the same directory.")
    st.stop()

# Define the preprocessing function
def preprocess_input(examen_admision, felder):
    # Create a DataFrame from user input
    input_df = pd.DataFrame({'Examen_admisión': [examen_admision], 'Felder': [felder]})

    # One-hot encode 'Felder'
    felder_encoded = onehot_encoder.transform(input_df[['Felder']])
    felder_feature_names = onehot_encoder.get_feature_names_out(['Felder'])
    felder_encoded_df = pd.DataFrame(felder_encoded, columns=felder_feature_names)

    # Scale 'Examen_admisión'
    examen_admision_scaled = minmax_scaler.transform(input_df[['Examen_admisión']])
    examen_admision_scaled_df = pd.DataFrame(examen_admision_scaled, columns=['Examen_admisión'])

    # Combine the features
    processed_input = pd.concat([examen_admision_scaled_df, felder_encoded_df], axis=1)

    return processed_input

# Streamlit UI
st.header('Ingrese los datos del estudiante')

examen_admision_input = st.number_input('Examen de Admisión', min_value=0.0, max_value=5.0, step=0.01)
felder_options = ['activo', 'visual', 'equilibrio', 'intuitivo', 'reflexivo', 'secuencial', 'sensorial', 'verbal'] # Replace with actual Felder categories
felder_input = st.selectbox('Estilo de Aprendizaje (Felder)', felder_options)


if st.button('Predecir Aprobación'):
    # Preprocess the input
    processed_data = preprocess_input(examen_admision_input, felder_input)

    # Make prediction
    prediction = best_stacking_model.predict(processed_data)

    # Display the prediction
    st.subheader('Resultado de la Predicción:')
    if prediction[0] == 'si':
        st.success('El estudiante probablemente APROBARÁ el curso.')
    else:
        st.error('El estudiante probablemente NO APROBARÁ el curso.')

st.header('Instrucciones para ejecutar la aplicación:')
st.write("1. Guarde este código como `app.py`.")
st.write("2. Asegúrese de tener los archivos `onehot_encoder.joblib`, `minmax_scaler.joblib`, y `best_stacking_model.joblib` en el mismo directorio.")
st.write("3. Abra una terminal en ese directorio.")
st.write("4. Ejecute el comando: `streamlit run app.py`")

Writing app.py
